# Generate Hoax News

## Member of Group
- Elisa Bayu Hendra
- Yohanes Emmanuel Putra Sutanto
- Federico Matthew Pratama
- Fernando Perry
- Vincentius Johanes Lwie Jaya
- Yohanes Bramanta Adita Saputra

# Versi 1 (The Best Choice)

## Preparation / Setup (Versi 1)

### Import Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import random
import re
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

## Full Code Generate Hoax (Versi 1 - Basic)

In [ ]:
class NewsChainGenerator:
    def __init__(self, use_gpu=True):
        """
        Initialize the news generator with AMD GPU support
        """
        # Check for GPU availability (works with AMD ROCm and NVIDIA CUDA)
        if use_gpu and torch.cuda.is_available():
            self.device = "cuda"
            print(f"Using GPU: {torch.cuda.get_device_name()}")
        elif use_gpu and hasattr(torch, 'xpu') and torch.xpu.is_available():
            self.device = "xpu"  # Intel GPU support
            print("Using Intel XPU")
        else:
            self.device = "cpu"
            print("Using CPU")

        # Load 20 newsgroups data
        print("Loading 20 Newsgroups dataset...")
        self.newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
        self.categories = self.newsgroups.target_names

        # Initialize components
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english',
                                        min_df=2, max_df=0.8)
        self.topic_model = LatentDirichletAllocation(n_components=20, random_state=42)

        # Markov chains for each category
        self.markov_chains = {}
        self.topic_keywords = {}

        # Initialize lightweight language model
        self.init_language_model()

    def init_language_model(self):
        """Initialize a lightweight language model for coherent text generation"""
        try:
            # Use DistilGPT2 for faster processing
            self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token

            # Initialize text generation pipeline with GPU support
            self.text_generator = pipeline(
                'text-generation',
                model='distilgpt2',
                tokenizer=self.tokenizer,
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )
            print("Language model initialized successfully")
        except Exception as e:
            print(f"Error initializing language model: {e}")
            self.text_generator = None

    def preprocess_text(self, text):
        """Clean and preprocess text"""
        # Remove email addresses, URLs, and special characters
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'http\S+|www\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip().lower()
        return text

    def build_markov_chains(self, order=2):
        """Build Markov chains for each news category"""
        print("Building Markov chains for each category...")

        for i, category in enumerate(self.categories):
            # Get texts for this category
            category_texts = [self.newsgroups.data[j] for j, target in enumerate(self.newsgroups.target) if target == i]

            # Combine and preprocess texts
            combined_text = ' '.join([self.preprocess_text(text) for text in category_texts[:100]])  # Limit for speed
            words = combined_text.split()

            if len(words) < order + 1:
                continue

            # Build Markov chain
            chain = defaultdict(list)
            for j in range(len(words) - order):
                key = tuple(words[j:j + order])
                next_word = words[j + order]
                chain[key].append(next_word)

            self.markov_chains[category] = dict(chain)
            print(f"Built chain for {category}: {len(chain)} states")

    def extract_topic_keywords(self):
        """Extract key topics and keywords using LDA"""
        print("Extracting topic keywords...")

        # Vectorize texts
        tfidf_matrix = self.vectorizer.fit_transform([self.preprocess_text(text) for text in self.newsgroups.data])

        # Fit topic model
        self.topic_model.fit(tfidf_matrix)

        # Extract keywords for each topic
        feature_names = self.vectorizer.get_feature_names_out()

        for topic_idx, topic in enumerate(self.topic_model.components_):
            top_keywords = [feature_names[i] for i in topic.argsort()[-10:]]
            self.topic_keywords[self.categories[topic_idx % len(self.categories)]] = top_keywords

        print("Topic extraction completed")

    def generate_with_markov(self, category, length=50):
        """Generate text using Markov chain"""
        if category not in self.markov_chains:
            return ""

        chain = self.markov_chains[category]
        if not chain:
            return ""

        # Start with random key
        current_key = random.choice(list(chain.keys()))
        result = list(current_key)

        for _ in range(length - len(current_key)):
            if current_key not in chain:
                break
            next_word = random.choice(chain[current_key])
            result.append(next_word)
            current_key = tuple(result[-2:])  # For order=2

        return ' '.join(result)

    def enhance_with_llm(self, seed_text, category, max_length=200):
        """Enhance text using language model for better coherence"""
        if not self.text_generator:
            return seed_text

        try:
            # Add category context
            prompt = f"News about {category.replace('_', ' ')}: {seed_text}"

            # Generate enhanced text
            generated = self.text_generator(
                prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.8,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

            return generated[0]['generated_text'].replace(prompt, '').strip()
        except Exception as e:
            print(f"LLM enhancement failed: {e}")
            return seed_text

    def generate_structured_article(self, category, target_length=300):
        """Generate a structured news article"""
        # Generate different parts using Markov chains
        headline = self.generate_with_markov(category, length=8)
        intro = self.generate_with_markov(category, length=30)
        body1 = self.generate_with_markov(category, length=40)
        body2 = self.generate_with_markov(category, length=40)
        conclusion = self.generate_with_markov(category, length=20)

        # Combine parts
        base_article = f"{headline}. {intro} {body1} {body2} {conclusion}"

        # Enhance with LLM if available
        if self.text_generator:
            enhanced_article = self.enhance_with_llm(base_article, category, target_length)
            return enhanced_article

        return base_article

    def generate_hoax_news(self, num_articles=5, categories=None):
        """Generate multiple hoax news articles"""
        if categories is None:
            categories = random.sample(self.categories, min(5, len(self.categories)))

        print(f"Generating {num_articles} articles across {len(categories)} categories...")

        articles = []
        for i in range(num_articles):
            category = random.choice(categories)

            print(f"Generating article {i+1}/{num_articles} for category: {category}")

            article = {
                'id': i + 1,
                'category': category,
                'title': self.generate_with_markov(category, length=6).title(),
                'content': self.generate_structured_article(category),
                'keywords': self.topic_keywords.get(category, [])[:5]
            }

            articles.append(article)

        return articles

    def train_and_generate(self, num_articles=5):
        """Main method to train models and generate articles"""
        print("Starting hoax news generation for research purposes...")
        print("=" * 60)

        # Build models
        self.build_markov_chains()
        self.extract_topic_keywords()

        # Generate articles
        articles = self.generate_hoax_news(num_articles)

        return articles

def main():
    """Main execution function"""
    print("Synthetic News Generator for Research")
    print("=" * 40)
    print("WARNING: This tool generates synthetic content for research purposes only!")
    print("=" * 40)

    # Initialize generator
    generator = NewsChainGenerator(use_gpu=True)

    # Generate articles
    articles = generator.train_and_generate(num_articles=3)  # Start with 3 for testing

    # Display results
    print("\n" + "=" * 60)
    print("GENERATED ARTICLES (FOR RESEARCH ONLY)")
    print("=" * 60)

    for article in articles:
        print(f"\nArticle {article['id']}")
        print(f"Category: {article['category']}")
        print(f"Title: {article['title']}")
        print(f"Keywords: {', '.join(article['keywords'])}")
        print("-" * 40)
        print(f"Content: {article['content'][:300]}...")
        print("-" * 60)

    # Save to file
    df = pd.DataFrame(articles)
    df.to_csv('synthetic_news_research.csv', index=False)
    print(f"\nResults saved to 'synthetic_news_research.csv'")

    return articles

if __name__ == "__main__":
    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)

    articles = main()

Synthetic News Generator for Research
Using CPU
Loading 20 Newsgroups dataset...


Device set to use cpu


Language model initialized successfully
Starting hoax news generation for research purposes...
Building Markov chains for each category...
Built chain for alt.atheism: 11899 states
Built chain for comp.graphics: 7868 states
Built chain for comp.os.ms-windows.misc: 7805 states
Built chain for comp.sys.ibm.pc.hardware: 8946 states
Built chain for comp.sys.mac.hardware: 6537 states
Built chain for comp.windows.x: 10717 states
Built chain for misc.forsale: 7354 states
Built chain for rec.autos: 9229 states
Built chain for rec.motorcycles: 8505 states
Built chain for rec.sport.baseball: 10053 states
Built chain for rec.sport.hockey: 9277 states
Built chain for sci.crypt: 17324 states
Built chain for sci.electronics: 9710 states
Built chain for sci.med: 10469 states
Built chain for sci.space: 15382 states
Built chain for soc.religion.christian: 16288 states
Built chain for talk.politics.guns: 13218 states
Built chain for talk.politics.mideast: 20442 states
Built chain for talk.politics.misc:

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Topic extraction completed
Generating 3 articles across 5 categories...
Generating article 1/3 for category: alt.atheism


Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating article 2/3 for category: alt.atheism


Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generating article 3/3 for category: talk.politics.guns

GENERATED ARTICLES (FOR RESEARCH ONLY)

Article 1
Category: alt.atheism
Title: Said Unto Them Ye Do Err
Keywords: file, using, email, program, know
----------------------------------------
Content: that what i am really being warned about is a religion in fact a religion with a vast majority of those who are atheists and for the purpose of promoting a religious liberty agenda are the ones who have also been the real world based on non-believers.
I think that if i were to say that this is a rel...
------------------------------------------------------------

Article 2
Category: alt.atheism
Title: The Lack Of Scepticism And So
Keywords: file, using, email, program, know
----------------------------------------
Content: of a fomenting state of barbarism and the human character's will to fight back against the evil of being a man. This is the same attitude as saying that the tl the nd a s is a sin, but that the nd a s is a sin, but t

### Explanation per Function

#### Initialization Phase

In [ ]:
class NewsChainGenerator:
    def __init__(self, use_gpu=True):
        """
        Initialize the news generator with AMD GPU support
        """
        # Check for GPU availability (works with AMD ROCm and NVIDIA CUDA)
        if use_gpu and torch.cuda.is_available():
            self.device = "cuda"
            print(f"Using GPU: {torch.cuda.get_device_name()}")
        elif use_gpu and hasattr(torch, 'xpu') and torch.xpu.is_available():
            self.device = "xpu"  # Intel GPU support
            print("Using Intel XPU")
        else:
            self.device = "cpu"
            print("Using CPU")

        # Load 20 newsgroups data
        print("Loading 20 Newsgroups dataset...")
        self.newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
        self.categories = self.newsgroups.target_names

        # Initialize components
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english',
                                        min_df=2, max_df=0.8)
        self.topic_model = LatentDirichletAllocation(n_components=20, random_state=42)

        # Markov chains for each category
        self.markov_chains = {}
        self.topic_keywords = {}

        # Initialize lightweight language model
        self.init_language_model()

#### Language Model Preparation

In [ ]:
def init_language_model(self):
        """Initialize a lightweight language model for coherent text generation"""
        try:
            # Use DistilGPT2 for faster processing
            self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token

            # Initialize text generation pipeline with GPU support
            self.text_generator = pipeline(
                'text-generation',
                model='distilgpt2',
                tokenizer=self.tokenizer,
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )
            print("Language model initialized successfully")
        except Exception as e:
            print(f"Error initializing language model: {e}")
            self.text_generator = None

#### Basic Text Cleaning & Normalization

In [ ]:
def preprocess_text(self, text):
        """Clean and preprocess text"""
        # Remove email addresses, URLs, and special characters
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'http\S+|www\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip().lower()
        return text

#### Markov Chain Model Training

In [ ]:
def build_markov_chains(self, order=2):
        """Build Markov chains for each news category"""
        print("Building Markov chains for each category...")

        for i, category in enumerate(self.categories):
            # Get texts for this category
            category_texts = [self.newsgroups.data[j] for j, target in enumerate(self.newsgroups.target) if target == i]

            # Combine and preprocess texts
            combined_text = ' '.join([self.preprocess_text(text) for text in category_texts[:100]])  # Limit for speed
            words = combined_text.split()

            if len(words) < order + 1:
                continue

            # Build Markov chain
            chain = defaultdict(list)
            for j in range(len(words) - order):
                key = tuple(words[j:j + order])
                next_word = words[j + order]
                chain[key].append(next_word)

            self.markov_chains[category] = dict(chain)
            print(f"Built chain for {category}: {len(chain)} states")

#### Topic Modeling & Keyword Extraction

In [ ]:
def extract_topic_keywords(self):
        """Extract key topics and keywords using LDA"""
        print("Extracting topic keywords...")

        # Vectorize texts
        tfidf_matrix = self.vectorizer.fit_transform([self.preprocess_text(text) for text in self.newsgroups.data])

        # Fit topic model
        self.topic_model.fit(tfidf_matrix)

        # Extract keywords for each topic
        feature_names = self.vectorizer.get_feature_names_out()

        for topic_idx, topic in enumerate(self.topic_model.components_):
            top_keywords = [feature_names[i] for i in topic.argsort()[-10:]]
            self.topic_keywords[self.categories[topic_idx % len(self.categories)]] = top_keywords

#### Basic Markov Chain Text Generation

In [ ]:
def generate_with_markov(self, category, length=50):
        """Generate text using Markov chain"""
        if category not in self.markov_chains:
            return ""

        chain = self.markov_chains[category]
        if not chain:
            return ""

        # Start with random key
        current_key = random.choice(list(chain.keys()))
        result = list(current_key)

        for _ in range(length - len(current_key)):
            if current_key not in chain:
                break
            next_word = random.choice(chain[current_key])
            result.append(next_word)
            current_key = tuple(result[-2:])  # For order=2

        return ' '.join(result)

#### LLM-Based Text Enhancement

In [ ]:
def enhance_with_llm(self, seed_text, category, max_length=200):
        """Enhance text using language model for better coherence"""
        if not self.text_generator:
            return seed_text

        try:
            # Add category context
            prompt = f"News about {category.replace('_', ' ')}: {seed_text}"

            # Generate enhanced text
            generated = self.text_generator(
                prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.8,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

            return generated[0]['generated_text'].replace(prompt, '').strip()
        except Exception as e:
            print(f"LLM enhancement failed: {e}")
            return seed_text

#### Structured Article Assembly

In [ ]:
def generate_structured_article(self, category, target_length=300):
        """Generate a structured news article"""
        # Generate different parts using Markov chains
        headline = self.generate_with_markov(category, length=8)
        intro = self.generate_with_markov(category, length=30)
        body1 = self.generate_with_markov(category, length=40)
        body2 = self.generate_with_markov(category, length=40)
        conclusion = self.generate_with_markov(category, length=20)

        # Combine parts
        base_article = f"{headline}. {intro} {body1} {body2} {conclusion}"

        # Enhance with LLM if available
        if self.text_generator:
            enhanced_article = self.enhance_with_llm(base_article, category, target_length)
            return enhanced_article

        return base_article

#### Multi-Article Batch Generation

In [ ]:
def generate_hoax_news(self, num_articles=5, categories=None):
        """Generate multiple hoax news articles"""
        if categories is None:
            categories = random.sample(self.categories, min(5, len(self.categories)))

        print(f"Generating {num_articles} articles across {len(categories)} categories...")

        articles = []
        for i in range(num_articles):
            category = random.choice(categories)

            print(f"Generating article {i+1}/{num_articles} for category: {category}")

            article = {
                'id': i + 1,
                'category': category,
                'title': self.generate_with_markov(category, length=6).title(),
                'content': self.generate_structured_article(category),
                'keywords': self.topic_keywords.get(category, [])[:5]
            }

            articles.append(article)

        return articles

#### Complete Pipeline Orchestration

In [ ]:
def train_and_generate(self, num_articles=5):
        """Main method to train models and generate articles"""
        print("Starting hoax news generation for research purposes...")
        print("=" * 60)

        # Build models
        self.build_markov_chains()
        self.extract_topic_keywords()

        # Generate articles
        articles = self.generate_hoax_news(num_articles)

        return articles

#### Main

In [ ]:
def main():
    """Main execution function"""
    print("Synthetic News Generator for Research")
    print("=" * 40)
    print("WARNING: This tool generates synthetic content for research purposes only!")
    print("=" * 40)

    # Initialize generator
    generator = NewsChainGenerator(use_gpu=True)

    # Generate articles
    articles = generator.train_and_generate(num_articles=3)  # Start with 3 for testing

    # Display results
    print("\n" + "=" * 60)
    print("GENERATED ARTICLES (FOR RESEARCH ONLY)")
    print("=" * 60)

    for article in articles:
        print(f"\nArticle {article['id']}")
        print(f"Category: {article['category']}")
        print(f"Title: {article['title']}")
        print(f"Keywords: {', '.join(article['keywords'])}")
        print("-" * 40)
        print(f"Content: {article['content'][:300]}...")
        print("-" * 60)

    # Save to file
    df = pd.DataFrame(articles)
    df.to_csv('synthetic_news_research.csv', index=False)
    print(f"\nResults saved to 'synthetic_news_research.csv'")

    return articles

if __name__ == "__main__":
    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)

    articles = main()

## Full Code Generate Hoax (Versi 1 - Advanced)

In [ ]:
class EnhancedNewsChainGenerator:
    def __init__(self, use_gpu=True, min_word_count=150):
        """
        Initialize the enhanced news generator with better filtering
        """
        # Check for GPU availability
        if use_gpu and torch.cuda.is_available():
            self.device = "cuda"
            print(f"Using GPU: {torch.cuda.get_device_name()}")
        elif use_gpu and hasattr(torch, 'xpu') and torch.xpu.is_available():
            self.device = "xpu"
            print("Using Intel XPU")
        else:
            self.device = "cpu"
            print("Using CPU")

        self.min_word_count = min_word_count
        print(f"Minimum word count per article: {min_word_count}")

        # Load 20 newsgroups data
        print("Loading 20 Newsgroups dataset...")
        self.newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
        self.categories = self.newsgroups.target_names

        # Initialize components
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english',
                                        min_df=2, max_df=0.8)
        self.topic_model = LatentDirichletAllocation(n_components=20, random_state=42)

        # Enhanced Markov chains
        self.markov_chains = {}
        self.word_frequencies = {}
        self.sentence_starters = {}
        self.sentence_enders = {}
        self.topic_keywords = {}

        # Common English words for validation
        self.common_words = set([
            'the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'it', 'for', 'not', 'on',
            'with', 'he', 'as', 'you', 'do', 'at', 'this', 'but', 'his', 'by', 'from', 'they',
            'we', 'say', 'her', 'she', 'or', 'an', 'will', 'my', 'one', 'all', 'would', 'there',
            'their', 'what', 'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go', 'me',
            'when', 'make', 'can', 'like', 'time', 'no', 'just', 'him', 'know', 'take', 'people',
            'into', 'year', 'your', 'good', 'some', 'could', 'them', 'see', 'other', 'than', 'then',
            'now', 'look', 'only', 'come', 'its', 'over', 'think', 'also', 'back', 'after', 'use',
            'two', 'how', 'our', 'work', 'first', 'well', 'way', 'even', 'new', 'want', 'because',
            'any', 'these', 'give', 'day', 'most', 'us', 'is', 'was', 'are', 'been', 'has', 'had',
            'were', 'said', 'each', 'which', 'did', 'very', 'where', 'much', 'too', 'own', 'while'
        ])

        # Initialize lightweight language model
        self.init_language_model()

    def init_language_model(self):
        """Initialize a lightweight language model for coherent text generation"""
        try:
            self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token

            self.text_generator = pipeline(
                'text-generation',
                model='distilgpt2',
                tokenizer=self.tokenizer,
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )
            print("Language model initialized successfully")
        except Exception as e:
            print(f"Error initializing language model: {e}")
            self.text_generator = None

    def is_valid_english_word(self, word):
        """Check if a word is likely to be a valid English word"""
        word = word.lower().strip()

        # Basic checks
        if len(word) < 2:
            return False

        # Check for common patterns that indicate non-English content
        suspicious_patterns = [
            r'[0-9]{3,}',  # Long numbers
            r'^[a-z]*[0-9]+[a-z]*[0-9]+',  # Mixed letters and numbers
            r'^[x-z]{2,}:',  # Protocol-like patterns (xs:, xx:)
            r'\.com|\.org|\.net|\.edu',  # Domain patterns
            r'^[a-f0-9]{8,}$',  # Hex strings
            r'[^a-z]',  # Non-alphabetic characters (after lowercasing)
        ]

        for pattern in suspicious_patterns:
            if re.search(pattern, word):
                return False

        # Check if it's a common English word or looks like English
        if word in self.common_words:
            return True

        # Check for basic English word patterns
        vowels = set('aeiou')
        consonants = set('bcdfghjklmnpqrstvwxyz')

        # Must contain at least one vowel (with some exceptions)
        if not any(c in vowels for c in word) and word not in ['by', 'my', 'fly', 'try', 'cry', 'dry', 'sky']:
            return False

        # Should not be all consonants or all vowels (with length > 3)
        if len(word) > 3:
            if all(c in consonants for c in word) or all(c in vowels for c in word):
                return False

        # Check for reasonable consonant/vowel distribution
        if len(word) > 6:
            vowel_count = sum(1 for c in word if c in vowels)
            if vowel_count < len(word) * 0.2:  # Less than 20% vowels
                return False

        return True

    def preprocess_text(self, text):
        """Clean and preprocess text with aggressive filtering"""
        # Remove email addresses, URLs, and suspicious patterns
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'http\S+|www\S+|ftp\S+', '', text)
        text = re.sub(r'[a-z]+://\S+', '', text)  # Any protocol
        text = re.sub(r'\S*\.\S*\.\S*', '', text)  # Multiple dots (likely addresses)
        text = re.sub(r'[0-9a-f]{8,}', '', text)  # Long hex strings
        text = re.sub(r'\b[a-z]*[0-9]+[a-z0-9]*\b', '', text)  # Mixed alphanumeric

        # Keep basic punctuation for sentence detection
        text = re.sub(r'[^a-zA-Z\s\.\!\?]', '', text)
        text = re.sub(r'\s+', ' ', text).strip().lower()

        # Filter out non-English words
        words = text.split()
        filtered_words = [word for word in words if self.is_valid_english_word(word)]

        return ' '.join(filtered_words)

    def build_enhanced_markov_chains(self, order=2):
        """Build enhanced Markov chains with aggressive filtering"""
        print("Building enhanced Markov chains for each category...")

        for i, category in enumerate(self.categories):
            print(f"Processing category: {category}")

            # Get texts for this category
            category_texts = [
                self.newsgroups.data[j]
                for j, target in enumerate(self.newsgroups.target)
                if target == i
            ][:100]  # Increase limit for better variety

            # Process texts and split into sentences
            all_sentences = []
            for text in category_texts:
                clean_text = self.preprocess_text(text)
                if len(clean_text.split()) < 5:  # Skip very short texts
                    continue

                sentences = [s.strip() for s in re.split(r'[.!?]+', clean_text) if len(s.strip()) > 15]

                # Additional filtering for sentences
                filtered_sentences = []
                for sentence in sentences:
                    words = sentence.split()
                    # Only keep sentences with reasonable English word ratio
                    valid_words = [w for w in words if self.is_valid_english_word(w)]
                    if len(valid_words) >= len(words) * 0.8 and len(valid_words) >= 5:
                        filtered_sentences.append(' '.join(valid_words))

                all_sentences.extend(filtered_sentences)

            if len(all_sentences) < 10:  # Need minimum sentences
                print(f"Insufficient clean sentences for {category}, skipping...")
                continue

            # Build chains, frequency maps, and sentence markers
            chain = defaultdict(lambda: defaultdict(int))
            word_freq = defaultdict(int)
            starters = []
            enders = set()

            for sentence in all_sentences[:300]:  # Increase limit
                words = sentence.split()
                if len(words) < order + 1:
                    continue

                # Track sentence starters
                if len(words) >= order:
                    starters.append(tuple(words[:order]))

                # Track sentence enders
                if len(words) >= 2:
                    enders.add(words[-1])

                # Build frequency-based chain
                for j in range(len(words) - order):
                    key = tuple(words[j:j + order])
                    next_word = words[j + order]

                    if self.is_valid_english_word(next_word):
                        chain[key][next_word] += 1
                        word_freq[next_word] += 1

            # Convert to probability distributions
            final_chain = {}
            for key, next_words in chain.items():
                if len(next_words) > 0:  # Only keep chains with valid next words
                    total = sum(next_words.values())
                    final_chain[key] = {
                        word: count/total
                        for word, count in next_words.items()
                    }

            self.markov_chains[category] = final_chain
            self.word_frequencies[category] = dict(word_freq)
            self.sentence_starters[category] = starters
            self.sentence_enders[category] = list(enders)

            print(f"Built chain for {category}: {len(final_chain)} states, {len(starters)} starters")

    def weighted_choice(self, choices_dict):
        """Make weighted random choice based on probabilities"""
        if not choices_dict:
            return None

        words = list(choices_dict.keys())
        weights = list(choices_dict.values())

        # Add some randomness to prevent always picking the most frequent word
        weights = np.array(weights)
        weights = np.power(weights, 0.7)  # Reduce dominance of most frequent words
        weights = weights / weights.sum()  # Normalize

        return np.random.choice(words, p=weights)

    def generate_with_improved_markov(self, category, max_length=50, target_sentences=3):
        """Generate text with improved Markov chain and loop detection"""
        if category not in self.markov_chains:
            return "Breaking news about recent developments in current affairs."

        chain = self.markov_chains[category]
        starters = self.sentence_starters.get(category, [])
        enders = self.sentence_enders.get(category, [])

        if not chain or not starters:
            return "Breaking news about recent developments in current affairs."

        result = []
        sentences_generated = 0
        used_sequences = set()  # Track used 4-grams to prevent loops
        attempts = 0
        max_attempts = 10

        while sentences_generated < target_sentences and len(result) < max_length and attempts < max_attempts:
            attempts += 1

            # Start new sentence
            current_key = random.choice(starters)
            sentence_words = list(current_key)

            # Generate sentence
            for _ in range(min(25, max_length - len(result))):  # Max 25 words per sentence
                if current_key not in chain:
                    break

                # Check for loops
                if len(sentence_words) >= 4:
                    recent_4gram = tuple(sentence_words[-4:])
                    if recent_4gram in used_sequences:
                        break
                    used_sequences.add(recent_4gram)

                # Choose next word using weighted selection
                next_word = self.weighted_choice(chain[current_key])
                if not next_word or not self.is_valid_english_word(next_word):
                    break

                sentence_words.append(next_word)

                # Check if we should end sentence
                if next_word in enders and len(sentence_words) > 6:
                    break
                elif len(sentence_words) > 20:  # Force sentence end
                    break

                # Update key for next iteration
                current_key = tuple(sentence_words[-2:])

            # Add sentence to result if it's substantial
            if len(sentence_words) > 5:  # Minimum sentence length
                result.extend(sentence_words)
                sentences_generated += 1

            # Clear old sequences periodically
            if len(used_sequences) > 30:
                used_sequences.clear()

        # Clean up result
        text = ' '.join(result[:max_length])
        text = re.sub(r'\s+', ' ', text).strip()

        # Ensure it ends properly
        if text and not text[-1] in '.!?':
            text += '.'

        return text

    def extract_topic_keywords(self):
        """Extract key topics and keywords using LDA"""
        print("Extracting topic keywords...")

        # Sample texts for faster processing
        sample_texts = random.sample(self.newsgroups.data, min(3000, len(self.newsgroups.data)))

        # Vectorize texts with better preprocessing
        processed_texts = []
        for text in sample_texts:
            clean_text = self.preprocess_text(text)
            if len(clean_text.split()) >= 10:  # Only use substantial texts
                processed_texts.append(clean_text)

        if len(processed_texts) < 100:
            print("Warning: Limited clean text available for topic extraction")
            return

        tfidf_matrix = self.vectorizer.fit_transform(processed_texts)

        # Fit topic model
        self.topic_model.fit(tfidf_matrix)

        # Extract keywords for each topic
        feature_names = self.vectorizer.get_feature_names_out()

        for topic_idx, topic in enumerate(self.topic_model.components_):
            # Get top keywords and filter for English words
            top_indices = topic.argsort()[-15:]  # Get more candidates
            top_keywords = [feature_names[i] for i in top_indices if self.is_valid_english_word(feature_names[i])]

            self.topic_keywords[self.categories[topic_idx % len(self.categories)]] = top_keywords[-8:]  # Keep top 8

        print("Topic extraction completed")

    def enhance_with_llm(self, seed_text, category, max_length=250):
        """Enhance text using language model for better coherence"""
        if not self.text_generator:
            return seed_text

        try:
            # Add category context
            prompt = f"News report about {category.replace('_', ' ')}: {seed_text[:80]}"

            # Generate enhanced text
            generated = self.text_generator(
                prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.3  # Stronger repetition penalty
            )

            enhanced = generated[0]['generated_text'].replace(prompt, '').strip()

            # Clean and filter the enhanced text
            sentences = enhanced.split('.')
            unique_sentences = []
            seen = set()

            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence not in seen and len(sentence) > 15:
                    # Check if sentence contains mostly English words
                    words = sentence.split()
                    valid_words = [w for w in words if self.is_valid_english_word(w.lower())]
                    if len(valid_words) >= len(words) * 0.8:
                        unique_sentences.append(sentence)
                        seen.add(sentence)
                if len(unique_sentences) >= 4:
                    break

            result = '. '.join(unique_sentences) + '.'
            return result if len(result.split()) >= 30 else seed_text

        except Exception as e:
            print(f"LLM enhancement failed: {e}")
            return seed_text

    def generate_structured_article(self, category, target_length=None):
        """Generate a structured news article with minimum word count"""
        if target_length is None:
            target_length = self.min_word_count

        # Generate different parts with controlled lengths
        intro = self.generate_with_improved_markov(category, max_length=35, target_sentences=2)
        body1 = self.generate_with_improved_markov(category, max_length=40, target_sentences=3)
        body2 = self.generate_with_improved_markov(category, max_length=35, target_sentences=2)
        body3 = self.generate_with_improved_markov(category, max_length=30, target_sentences=2)

        # Combine parts with transitions
        transitions = ["Meanwhile", "Furthermore", "Additionally", "However", "In related developments", "Moreover"]
        transition1 = random.choice(transitions)
        transition2 = random.choice([t for t in transitions if t != transition1])

        base_article = f"{intro} {transition1.lower()}, {body1} {transition2.lower()}, {body2} {body3}"

        # Clean up the article
        base_article = re.sub(r'\s+', ' ', base_article)
        base_article = re.sub(r'\.+', '.', base_article)
        base_article = base_article.replace(' .', '.')

        # Check word count and extend if necessary
        current_word_count = len(base_article.split())

        if current_word_count < self.min_word_count:
            # Generate additional content
            additional_needed = self.min_word_count - current_word_count
            extra_content = self.generate_with_improved_markov(
                category,
                max_length=additional_needed + 20,
                target_sentences=max(2, additional_needed // 15)
            )
            base_article += f" In conclusion, {extra_content}"

        # Enhance with LLM if available
        if self.text_generator and len(base_article.split()) < target_length:
            enhanced_article = self.enhance_with_llm(base_article, category, target_length + 50)
            final_word_count = len(enhanced_article.split())

            if final_word_count >= self.min_word_count:
                return enhanced_article

        return base_article

    def generate_hoax_news(self, num_articles=5, categories=None):
        """Generate multiple synthetic news articles with minimum word count"""
        if categories is None:
            # Filter categories that have sufficient data
            valid_categories = [cat for cat in self.categories if cat in self.markov_chains]
            if len(valid_categories) < 5:
                categories = valid_categories
            else:
                categories = random.sample(valid_categories, min(8, len(valid_categories)))

        print(f"Generating {num_articles} articles across {len(categories)} categories...")

        articles = []
        for i in range(num_articles):
            category = random.choice(categories)

            print(f"Generating article {i+1}/{num_articles} for category: {category}")

            # Generate title separately
            title_text = self.generate_with_improved_markov(category, max_length=8, target_sentences=1)
            title_words = [w for w in title_text.split() if self.is_valid_english_word(w)]
            title = ' '.join(title_words[:7]).replace('.', '').title()

            # Generate content
            content = self.generate_structured_article(category)
            word_count = len(content.split())

            article = {
                'id': i + 1,
                'category': category.replace('_', ' ').title(),
                'title': title if title else f"News Report {i+1}",
                'content': content,
                'word_count': word_count,
                'keywords': self.topic_keywords.get(category, [])[:5]
            }

            articles.append(article)
            print(f"✓ Generated article with {word_count} words")

        return articles

    def train_and_generate(self, num_articles=3):
        """Main method to train models and generate articles"""
        print("Starting enhanced news generation for research purposes...")
        print(f"Minimum word count per article: {self.min_word_count}")
        print("=" * 60)

        # Build improved models
        self.build_enhanced_markov_chains()
        self.extract_topic_keywords()

        # Generate articles
        articles = self.generate_hoax_news(num_articles)

        return articles

def main():
    """Main execution function"""
    print("Enhanced Synthetic News Generator for Research")
    print("=" * 55)
    print("WARNING: This tool generates synthetic content for research purposes only!")
    print("Features: Advanced filtering, minimum word count, English-only content")
    print("=" * 55)

    # Initialize generator with minimum word count
    generator = EnhancedNewsChainGenerator(use_gpu=True, min_word_count=300)

    # Generate articles
    articles = generator.train_and_generate(num_articles=5)

    # Display results
    print("\n" + "=" * 70)
    print("GENERATED ARTICLES (FOR RESEARCH ONLY)")
    print("=" * 70)

    total_words = 0
    for article in articles:
        print(f"\n📰 Article {article['id']}")
        print(f"📂 Category: {article['category']}")
        print(f"📰 Title: {article['title']}")
        print(f"📊 Word Count: {article['word_count']} words")
        print(f"🏷️  Keywords: {', '.join(article['keywords'])}")
        print("-" * 60)
        print(f"📄 Content: {article['content']}")
        print("-" * 70)
        total_words += article['word_count']

    print(f"\n📈 Total words generated: {total_words}")
    print(f"📈 Average words per article: {total_words/len(articles):.1f}")

    # Save to file
    df = pd.DataFrame(articles)
    df.to_csv('enhanced_synthetic_news_research.csv', index=False)
    print(f"\n💾 Results saved to 'enhanced_synthetic_news_research.csv'")

    return articles

if __name__ == "__main__":
    # Set random seeds for reproducibility
    # random.seed(42)
    # np.random.seed(42)
    # torch.manual_seed(42)

    articles = main()

Enhanced Synthetic News Generator for Research
Features: Advanced filtering, minimum word count, English-only content
Using CPU
Minimum word count per article: 300
Loading 20 Newsgroups dataset...


Device set to use cpu


Language model initialized successfully
Starting enhanced news generation for research purposes...
Minimum word count per article: 300
Building enhanced Markov chains for each category...
Processing category: alt.atheism
Built chain for alt.atheism: 10553 states, 95 starters
Processing category: comp.graphics
Built chain for comp.graphics: 6854 states, 97 starters
Processing category: comp.os.ms-windows.misc
Built chain for comp.os.ms-windows.misc: 6310 states, 94 starters
Processing category: comp.sys.ibm.pc.hardware
Built chain for comp.sys.ibm.pc.hardware: 7733 states, 99 starters
Processing category: comp.sys.mac.hardware
Built chain for comp.sys.mac.hardware: 5656 states, 98 starters
Processing category: comp.windows.x
Built chain for comp.windows.x: 9139 states, 97 starters
Processing category: misc.forsale
Built chain for misc.forsale: 6302 states, 97 starters
Processing category: rec.autos
Built chain for rec.autos: 8041 states, 96 starters
Processing category: rec.motorcycles


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=350) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Topic extraction completed
Generating 5 articles across 8 categories...
Generating article 1/5 for category: comp.sys.ibm.pc.hardware


...

✓ Generated article with 228 words
Generating article 3/5 for category: talk.religion.misc


Both `max_new_tokens` (=256) and `max_length`(=350) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✓ Generated article with 161 words
Generating article 4/5 for category: sci.electronics


Both `max_new_tokens` (=256) and `max_length`(=350) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✓ Generated article with 169 words
Generating article 5/5 for category: sci.crypt
✓ Generated article with 153 words

GENERATED ARTICLES (FOR RESEARCH ONLY)

📰 Article 1
📂 Category: Comp.Sys.Ibm.Pc.Hardware
📰 Title: Posted This Once But Didnt Even
📊 Word Count: 207 words
🏷️  Keywords: ibm, hayes, lebanese, cards, fault
------------------------------------------------------------
📄 Content: the adaptec is best value for dont think the is it possible through either pin configuration or through software programming to. however, have tried numerous combinations to no they work alone or can anyone tell me for the video and thanks to orchid got fix from their tech support hi all netters if upgrade my to. furthermore, that figure for image rotation would seem to hello all you the net are my. hello all you the net are my because of some help with the stealth get. In conclusion, today recieved inwarranty replacement for my diamond speedstar on the its not so easy to exchange these things the on

### Explanation per Function

#### Initialization Phase

In [ ]:
class EnhancedNewsChainGenerator:
    def __init__(self, use_gpu=True, min_word_count=150):
        """
        Initialize the enhanced news generator with better filtering
        """
        # Check for GPU availability
        if use_gpu and torch.cuda.is_available():
            self.device = "cuda"
            print(f"Using GPU: {torch.cuda.get_device_name()}")
        elif use_gpu and hasattr(torch, 'xpu') and torch.xpu.is_available():
            self.device = "xpu"
            print("Using Intel XPU")
        else:
            self.device = "cpu"
            print("Using CPU")

        self.min_word_count = min_word_count
        print(f"Minimum word count per article: {min_word_count}")

        # Load 20 newsgroups data
        print("Loading 20 Newsgroups dataset...")
        self.newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
        self.categories = self.newsgroups.target_names

        # Initialize components
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english',
                                        min_df=2, max_df=0.8)
        self.topic_model = LatentDirichletAllocation(n_components=20, random_state=42)

        # Enhanced Markov chains
        self.markov_chains = {}
        self.word_frequencies = {}
        self.sentence_starters = {}
        self.sentence_enders = {}
        self.topic_keywords = {}

        # Common English words for validation
        self.common_words = set([
            'the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'it', 'for', 'not', 'on',
            'with', 'he', 'as', 'you', 'do', 'at', 'this', 'but', 'his', 'by', 'from', 'they',
            'we', 'say', 'her', 'she', 'or', 'an', 'will', 'my', 'one', 'all', 'would', 'there',
            'their', 'what', 'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go', 'me',
            'when', 'make', 'can', 'like', 'time', 'no', 'just', 'him', 'know', 'take', 'people',
            'into', 'year', 'your', 'good', 'some', 'could', 'them', 'see', 'other', 'than', 'then',
            'now', 'look', 'only', 'come', 'its', 'over', 'think', 'also', 'back', 'after', 'use',
            'two', 'how', 'our', 'work', 'first', 'well', 'way', 'even', 'new', 'want', 'because',
            'any', 'these', 'give', 'day', 'most', 'us', 'is', 'was', 'are', 'been', 'has', 'had',
            'were', 'said', 'each', 'which', 'did', 'very', 'where', 'much', 'too', 'own', 'while'
        ])

        # Initialize lightweight language model
        self.init_language_model()

#### Language Model Preparation

In [ ]:
def init_language_model(self):
        """Initialize a lightweight language model for coherent text generation"""
        try:
            self.tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token

            self.text_generator = pipeline(
                'text-generation',
                model='distilgpt2',
                tokenizer=self.tokenizer,
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )
            print("Language model initialized successfully")
        except Exception as e:
            print(f"Error initializing language model: {e}")
            self.text_generator = None

#### English Word Validation Engine

In [ ]:
def is_valid_english_word(self, word):
        """Check if a word is likely to be a valid English word"""
        word = word.lower().strip()

        # Basic checks
        if len(word) < 2:
            return False

        # Check for common patterns that indicate non-English content
        suspicious_patterns = [
            r'[0-9]{3,}',  # Long numbers
            r'^[a-z]*[0-9]+[a-z]*[0-9]+',  # Mixed letters and numbers
            r'^[x-z]{2,}:',  # Protocol-like patterns (xs:, xx:)
            r'\.com|\.org|\.net|\.edu',  # Domain patterns
            r'^[a-f0-9]{8,}$',  # Hex strings
            r'[^a-z]',  # Non-alphabetic characters (after lowercasing)
        ]

        for pattern in suspicious_patterns:
            if re.search(pattern, word):
                return False

        # Check if it's a common English word or looks like English
        if word in self.common_words:
            return True

        # Check for basic English word patterns
        vowels = set('aeiou')
        consonants = set('bcdfghjklmnpqrstvwxyz')

        # Must contain at least one vowel (with some exceptions)
        if not any(c in vowels for c in word) and word not in ['by', 'my', 'fly', 'try', 'cry', 'dry', 'sky']:
            return False

        # Should not be all consonants or all vowels (with length > 3)
        if len(word) > 3:
            if all(c in consonants for c in word) or all(c in vowels for c in word):
                return False

        # Check for reasonable consonant/vowel distribution
        if len(word) > 6:
            vowel_count = sum(1 for c in word if c in vowels)
            if vowel_count < len(word) * 0.2:  # Less than 20% vowels
                return False

        return True

#### Advanced Text Cleaning & Aggressive Filtering

In [ ]:
def preprocess_text(self, text):
        """Clean and preprocess text with aggressive filtering"""
        # Remove email addresses, URLs, and suspicious patterns
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'http\S+|www\S+|ftp\S+', '', text)
        text = re.sub(r'[a-z]+://\S+', '', text)  # Any protocol
        text = re.sub(r'\S*\.\S*\.\S*', '', text)  # Multiple dots (likely addresses)
        text = re.sub(r'[0-9a-f]{8,}', '', text)  # Long hex strings
        text = re.sub(r'\b[a-z]*[0-9]+[a-z0-9]*\b', '', text)  # Mixed alphanumeric

        # Keep basic punctuation for sentence detection
        text = re.sub(r'[^a-zA-Z\s\.\!\?]', '', text)
        text = re.sub(r'\s+', ' ', text).strip().lower()

        # Filter out non-English words
        words = text.split()
        filtered_words = [word for word in words if self.is_valid_english_word(word)]

        return ' '.join(filtered_words)

#### Enhanced Markov Chain Model Training with Probabilistic Weighting

In [ ]:
def build_enhanced_markov_chains(self, order=2):
        """Build enhanced Markov chains with aggressive filtering"""
        print("Building enhanced Markov chains for each category...")

        for i, category in enumerate(self.categories):
            print(f"Processing category: {category}")

            # Get texts for this category
            category_texts = [
                self.newsgroups.data[j]
                for j, target in enumerate(self.newsgroups.target)
                if target == i
            ][:100]  # Increase limit for better variety

            # Process texts and split into sentences
            all_sentences = []
            for text in category_texts:
                clean_text = self.preprocess_text(text)
                if len(clean_text.split()) < 5:  # Skip very short texts
                    continue

                sentences = [s.strip() for s in re.split(r'[.!?]+', clean_text) if len(s.strip()) > 15]

                # Additional filtering for sentences
                filtered_sentences = []
                for sentence in sentences:
                    words = sentence.split()
                    # Only keep sentences with reasonable English word ratio
                    valid_words = [w for w in words if self.is_valid_english_word(w)]
                    if len(valid_words) >= len(words) * 0.8 and len(valid_words) >= 5:
                        filtered_sentences.append(' '.join(valid_words))

                all_sentences.extend(filtered_sentences)

            if len(all_sentences) < 10:  # Need minimum sentences
                print(f"Insufficient clean sentences for {category}, skipping...")
                continue

            # Build chains, frequency maps, and sentence markers
            chain = defaultdict(lambda: defaultdict(int))
            word_freq = defaultdict(int)
            starters = []
            enders = set()

            for sentence in all_sentences[:300]:  # Increase limit
                words = sentence.split()
                if len(words) < order + 1:
                    continue

                # Track sentence starters
                if len(words) >= order:
                    starters.append(tuple(words[:order]))

                # Track sentence enders
                if len(words) >= 2:
                    enders.add(words[-1])

                # Build frequency-based chain
                for j in range(len(words) - order):
                    key = tuple(words[j:j + order])
                    next_word = words[j + order]

                    if self.is_valid_english_word(next_word):
                        chain[key][next_word] += 1
                        word_freq[next_word] += 1

            # Convert to probability distributions
            final_chain = {}
            for key, next_words in chain.items():
                if len(next_words) > 0:  # Only keep chains with valid next words
                    total = sum(next_words.values())
                    final_chain[key] = {
                        word: count/total
                        for word, count in next_words.items()
                    }

            self.markov_chains[category] = final_chain
            self.word_frequencies[category] = dict(word_freq)
            self.sentence_starters[category] = starters
            self.sentence_enders[category] = list(enders)

            print(f"Built chain for {category}: {len(final_chain)} states, {len(starters)} starters")

#### Weighted Random Selection for Natural Text Flow

In [ ]:
def weighted_choice(self, choices_dict):
        """Make weighted random choice based on probabilities"""
        if not choices_dict:
            return None

        words = list(choices_dict.keys())
        weights = list(choices_dict.values())

        # Add some randomness to prevent always picking the most frequent word
        weights = np.array(weights)
        weights = np.power(weights, 0.7)  # Reduce dominance of most frequent words
        weights = weights / weights.sum()  # Normalize

        return np.random.choice(words, p=weights)

#### Enhanced Topic Modeling & Keyword Extraction

In [ ]:
def extract_topic_keywords(self):
        """Extract key topics and keywords using LDA"""
        print("Extracting topic keywords...")

        # Sample texts for faster processing
        sample_texts = random.sample(self.newsgroups.data, min(3000, len(self.newsgroups.data)))

        # Vectorize texts with better preprocessing
        processed_texts = []
        for text in sample_texts:
            clean_text = self.preprocess_text(text)
            if len(clean_text.split()) >= 10:  # Only use substantial texts
                processed_texts.append(clean_text)

        if len(processed_texts) < 100:
            print("Warning: Limited clean text available for topic extraction")
            return

        tfidf_matrix = self.vectorizer.fit_transform(processed_texts)

        # Fit topic model
        self.topic_model.fit(tfidf_matrix)

        # Extract keywords for each topic
        feature_names = self.vectorizer.get_feature_names_out()

        for topic_idx, topic in enumerate(self.topic_model.components_):
            # Get top keywords and filter for English words
            top_indices = topic.argsort()[-15:]  # Get more candidates
            top_keywords = [feature_names[i] for i in top_indices if self.is_valid_english_word(feature_names[i])]

            self.topic_keywords[self.categories[topic_idx % len(self.categories)]] = top_keywords[-8:]  # Keep top 8

        print("Topic extraction completed")

#### Advanced Markov Chain Text Generation with Loop Detection

In [ ]:
def generate_with_improved_markov(self, category, max_length=50, target_sentences=3):
        """Generate text with improved Markov chain and loop detection"""
        if category not in self.markov_chains:
            return "Breaking news about recent developments in current affairs."

        chain = self.markov_chains[category]
        starters = self.sentence_starters.get(category, [])
        enders = self.sentence_enders.get(category, [])

        if not chain or not starters:
            return "Breaking news about recent developments in current affairs."

        result = []
        sentences_generated = 0
        used_sequences = set()  # Track used 4-grams to prevent loops
        attempts = 0
        max_attempts = 10

        while sentences_generated < target_sentences and len(result) < max_length and attempts < max_attempts:
            attempts += 1

            # Start new sentence
            current_key = random.choice(starters)
            sentence_words = list(current_key)

            # Generate sentence
            for _ in range(min(25, max_length - len(result))):  # Max 25 words per sentence
                if current_key not in chain:
                    break

                # Check for loops
                if len(sentence_words) >= 4:
                    recent_4gram = tuple(sentence_words[-4:])
                    if recent_4gram in used_sequences:
                        break
                    used_sequences.add(recent_4gram)

                # Choose next word using weighted selection
                next_word = self.weighted_choice(chain[current_key])
                if not next_word or not self.is_valid_english_word(next_word):
                    break

                sentence_words.append(next_word)

                # Check if we should end sentence
                if next_word in enders and len(sentence_words) > 6:
                    break
                elif len(sentence_words) > 20:  # Force sentence end
                    break

                # Update key for next iteration
                current_key = tuple(sentence_words[-2:])

            # Add sentence to result if it's substantial
            if len(sentence_words) > 5:  # Minimum sentence length
                result.extend(sentence_words)
                sentences_generated += 1

            # Clear old sequences periodically
            if len(used_sequences) > 30:
                used_sequences.clear()

        # Clean up result
        text = ' '.join(result[:max_length])
        text = re.sub(r'\s+', ' ', text).strip()

        # Ensure it ends properly
        if text and not text[-1] in '.!?':
            text += '.'

        return text

#### Enhanced LLM-Based Text Enhancement with Deduplication

In [ ]:
def enhance_with_llm(self, seed_text, category, max_length=250):
        """Enhance text using language model for better coherence"""
        if not self.text_generator:
            return seed_text

        try:
            # Add category context
            prompt = f"News report about {category.replace('_', ' ')}: {seed_text[:80]}"

            # Generate enhanced text
            generated = self.text_generator(
                prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.3  # Stronger repetition penalty
            )

            enhanced = generated[0]['generated_text'].replace(prompt, '').strip()

            # Clean and filter the enhanced text
            sentences = enhanced.split('.')
            unique_sentences = []
            seen = set()

            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence not in seen and len(sentence) > 15:
                    # Check if sentence contains mostly English words
                    words = sentence.split()
                    valid_words = [w for w in words if self.is_valid_english_word(w.lower())]
                    if len(valid_words) >= len(words) * 0.8:
                        unique_sentences.append(sentence)
                        seen.add(sentence)
                if len(unique_sentences) >= 4:
                    break

            result = '. '.join(unique_sentences) + '.'
            return result if len(result.split()) >= 30 else seed_text

        except Exception as e:
            print(f"LLM enhancement failed: {e}")
            return seed_text

#### Structured Article Assembly with Minimum Word Count Control

In [ ]:
def generate_structured_article(self, category, target_length=None):
        """Generate a structured news article with minimum word count"""
        if target_length is None:
            target_length = self.min_word_count

        # Generate different parts with controlled lengths
        intro = self.generate_with_improved_markov(category, max_length=35, target_sentences=2)
        body1 = self.generate_with_improved_markov(category, max_length=40, target_sentences=3)
        body2 = self.generate_with_improved_markov(category, max_length=35, target_sentences=2)
        body3 = self.generate_with_improved_markov(category, max_length=30, target_sentences=2)

        # Combine parts with transitions
        transitions = ["Meanwhile", "Furthermore", "Additionally", "However", "In related developments", "Moreover"]
        transition1 = random.choice(transitions)
        transition2 = random.choice([t for t in transitions if t != transition1])

        base_article = f"{intro} {transition1.lower()}, {body1} {transition2.lower()}, {body2} {body3}"

        # Clean up the article
        base_article = re.sub(r'\s+', ' ', base_article)
        base_article = re.sub(r'\.+', '.', base_article)
        base_article = base_article.replace(' .', '.')

        # Check word count and extend if necessary
        current_word_count = len(base_article.split())

        if current_word_count < self.min_word_count:
            # Generate additional content
            additional_needed = self.min_word_count - current_word_count
            extra_content = self.generate_with_improved_markov(
                category,
                max_length=additional_needed + 20,
                target_sentences=max(2, additional_needed // 15)
            )
            base_article += f" In conclusion, {extra_content}"

        # Enhance with LLM if available
        if self.text_generator and len(base_article.split()) < target_length:
            enhanced_article = self.enhance_with_llm(base_article, category, target_length + 50)
            final_word_count = len(enhanced_article.split())

            if final_word_count >= self.min_word_count:
                return enhanced_article

        return base_article

#### Multi-Article Batch Generation with Quality Control

In [ ]:
def generate_hoax_news(self, num_articles=5, categories=None):
        """Generate multiple synthetic news articles with minimum word count"""
        if categories is None:
            # Filter categories that have sufficient data
            valid_categories = [cat for cat in self.categories if cat in self.markov_chains]
            if len(valid_categories) < 5:
                categories = valid_categories
            else:
                categories = random.sample(valid_categories, min(8, len(valid_categories)))

        print(f"Generating {num_articles} articles across {len(categories)} categories...")

        articles = []
        for i in range(num_articles):
            category = random.choice(categories)

            print(f"Generating article {i+1}/{num_articles} for category: {category}")

            # Generate title separately
            title_text = self.generate_with_improved_markov(category, max_length=8, target_sentences=1)
            title_words = [w for w in title_text.split() if self.is_valid_english_word(w)]
            title = ' '.join(title_words[:7]).replace('.', '').title()

            # Generate content
            content = self.generate_structured_article(category)
            word_count = len(content.split())

            article = {
                'id': i + 1,
                'category': category.replace('_', ' ').title(),
                'title': title if title else f"News Report {i+1}",
                'content': content,
                'word_count': word_count,
                'keywords': self.topic_keywords.get(category, [])[:5]
            }

            articles.append(article)
            print(f"✓ Generated article with {word_count} words")

        return articles

#### Complete Enhanced Pipeline Orchestration

In [ ]:
def train_and_generate(self, num_articles=3):
        """Main method to train models and generate articles"""
        print("Starting enhanced news generation for research purposes...")
        print(f"Minimum word count per article: {self.min_word_count}")
        print("=" * 60)

        # Build improved models
        self.build_enhanced_markov_chains()
        self.extract_topic_keywords()

        # Generate articles
        articles = self.generate_hoax_news(num_articles)

        return articles

#### Main

In [ ]:
def main():
    """Main execution function"""
    print("Enhanced Synthetic News Generator for Research")
    print("=" * 55)
    print("WARNING: This tool generates synthetic content for research purposes only!")
    print("Features: Advanced filtering, minimum word count, English-only content")
    print("=" * 55)

    # Initialize generator with minimum word count
    generator = EnhancedNewsChainGenerator(use_gpu=True, min_word_count=300)

    # Generate articles
    articles = generator.train_and_generate(num_articles=5)

    # Display results
    print("\n" + "=" * 70)
    print("GENERATED ARTICLES (FOR RESEARCH ONLY)")
    print("=" * 70)

    total_words = 0
    for article in articles:
        print(f"\n📰 Article {article['id']}")
        print(f"📂 Category: {article['category']}")
        print(f"📰 Title: {article['title']}")
        print(f"📊 Word Count: {article['word_count']} words")
        print(f"🏷️  Keywords: {', '.join(article['keywords'])}")
        print("-" * 60)
        print(f"📄 Content: {article['content']}")
        print("-" * 70)
        total_words += article['word_count']

    print(f"\n📈 Total words generated: {total_words}")
    print(f"📈 Average words per article: {total_words/len(articles):.1f}")

    # Save to file
    df = pd.DataFrame(articles)
    df.to_csv('enhanced_synthetic_news_research.csv', index=False)
    print(f"\n💾 Results saved to 'enhanced_synthetic_news_research.csv'")

    return articles

if __name__ == "__main__":
    # Set random seeds for reproducibility
    # random.seed(42)
    # np.random.seed(42)
    # torch.manual_seed(42)

    articles = main()

# Versi 2 (OK)

## Preparation / Setup (Versi 2)

### Import Library and Setup Database

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import random
import re
import spacy
from nltk.tokenize import sent_tokenize
import nltk
from collections import defaultdict

# Download required NLTK data
nltk.download('punkt')

# Load spaCy model for better NLP processing
try:
    nlp = spacy.load('en_core_web_sm')
except:
    import sys
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load('en_core_web_sm')

# Load the 20 newsgroups dataset with more relevant categories for news
categories = [
    'talk.politics.misc',
    'talk.politics.guns',
    'talk.politics.mideast',
    'sci.med',
    'sci.space',
    'comp.sys.ibm.pc.hardware',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.electronics'
]

newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    remove=('headers', 'footers', 'quotes')
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Function for Cleaning Text

In [ ]:
# Improved cleaning function
def clean_text(text):
    # Remove email addresses, URLs and normalize whitespace
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)

    # Remove common artifacts from newsgroups data
    text = re.sub(r'writes:', '', text)
    text = re.sub(r'wrote:', '', text)
    text = re.sub(r'saying:', '', text)
    text = re.sub(r'>\s*>', '', text)
    text = re.sub(r'>', '', text)

    # Remove non-standard characters
    text = re.sub(r'[^\w\s.,;:!?\'"-]', '', text)

    # Remove lines that are too short (likely fragments)
    lines = text.split('\n')
    lines = [line for line in lines if len(line.strip().split()) > 3]
    text = ' '.join(lines)

    return text.strip()

print("Cleaning texts...")
cleaned_texts = [clean_text(text) for text in newsgroups.data]
cleaned_texts = [text for text in cleaned_texts if len(text) > 100]

# Print 10 examples of cleaned results
print(f"\nTotal cleaned texts: {len(cleaned_texts)}")
print("\n" + "="*80)
print("EXAMPLES OF CLEANED TEXTS (First 5):")
print("="*80)

for i, text in enumerate(cleaned_texts[:5], 1):
    print(f"\n--- Example {i} ---")
    # Print first 300 characters to keep it readable
    print(text[:300] + "..." if len(text) > 300 else text)
    print(f"Length: {len(text)} characters")

Cleaning texts...

Total cleaned texts: 7761

EXAMPLES OF CLEANED TEXTS (First 5):

--- Example 1 ---
misc.legal trimmed Well, that's the obvious conclusion, given your train of logic. The corollary then is that it must be a waste of time for the party to run candidates until the educational program has shown some results. Followups to a.p.l. --
Length: 245 characters

--- Example 2 ---
You can configure devices for the same IRQ as long as you don't use them simultaneously, under Dos at least. Both LPT1: and SB just sit there until you tell them to do something. You can't configure a SoundBlaster for IRQ7 if you got an Ethernet Card which hits that IRQ a thousand times or so per se...
Length: 305 characters

--- Example 3 ---
If Croats are now divided, it is because Croatia seceded from Yugoslavia. Croats in Croatia, B-H, and Serbia were in one country--Yugoslavia-- until they divided themselves. If Muslims are now divided, it is because B-H seceded from Yugoslavia. Muslims in Croatia,

## Full Code Generate Hoax (Versi 2 - Basic)

In [ ]:
print("Extracting sentences...")
all_sentences = []
for text in cleaned_texts:
    doc = nlp(text)
    for sent in doc.sents:
        sent_text = sent.text.strip()
        # Filter sentences that are complete and of reasonable length
        if len(sent_text.split()) >= 5 and len(sent_text.split()) <= 30:
            if sent_text[-1] in '.!?':
                if not re.search(r'(^\W|^\d+\.|^>|^I think|^I believe|^However|^But|^And|^Or|^So,|^Also,)', sent_text):
                    all_sentences.append(sent_text)

print("Vectorizing sentences...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2,
    max_features=1000,
    stop_words='english'
)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

print("Fitting LDA model...")
n_topics = 15
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42,
    batch_size=128,
    n_jobs=-1
)
lda_output = lda.fit_transform(tfidf_matrix)
topic_assignments = lda_output.argmax(axis=1)

sentences_by_topic = defaultdict(list)
for i, topic_idx in enumerate(topic_assignments):
    sentences_by_topic[topic_idx].append(all_sentences[i])

# Extract keywords for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
keywords_by_topic = {}

for topic_idx, topic in enumerate(lda.components_):
    top_indices = topic.argsort()[:-11:-1]  # Get top 10 keywords
    top_keywords = [feature_names[i] for i in top_indices]
    keywords_by_topic[topic_idx] = top_keywords

# Define topic names based on keywords
topic_names = {}
for topic_idx, keywords in keywords_by_topic.items():
    topic_names[topic_idx] = ' '.join(keywords[:3])

# Create entity database for more realistic entity mentions
entity_types = {
    'PERSON': set(),
    'ORG': set(),
    'GPE': set(),  # Countries, cities
    'DATE': set()
}

print("Extracting entities...")
# Process a subset to save time
for text in random.sample(cleaned_texts, min(1000, len(cleaned_texts))):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entity_types and len(ent.text) > 1:
            entity_types[ent.label_].add(ent.text)

# Template sentences for different parts of an article
templates = {
    'headline': [
        "BREAKING: {topic} - {keyword} {keyword2} Raises Concerns",
        "ALERT: New Developments in {topic} as {org} Announces {keyword}",
        "EXCLUSIVE: {person} Reveals Critical Information About {topic}",
        "{topic} CRISIS: What {org} Isn't Telling You About {keyword}",
        "JUST IN: {keyword} {keyword2} Scandal Rocks {org}"
    ],
    'intro': [
        "A recent development regarding {topic} has sparked widespread debate.",
        "New information has emerged about {topic} that demands attention.",
        "In a stunning revelation, details about {topic} have come to light.",
        "Reports are circulating about unprecedented events related to {topic}."
    ],
    'quote': [
        "\"{sentence}\" said {person}, who {verb} the {noun}.",
        "\"{sentence}\" according to {person} from {org}.",
        "{person} stated, \"{sentence}\" This comes after {org} {verb} the {noun}.",
        "\"{sentence}\" {person} explained during the recent {event}."
    ],
    'transition': [
        "Meanwhile, other sources indicate that",
        "This development comes at a time when",
        "Experts suggest that this could mean",
        "Analysis of the situation reveals",
        "Critics argue that these events demonstrate"
    ]
}

verbs = ["addressed", "questioned", "investigated", "analyzed", "evaluated",
         "criticized", "supported", "endorsed", "reviewed", "examined"]
nouns = ["situation", "statement", "report", "findings", "allegations",
         "evidence", "controversy", "development", "policy", "decision"]
events = ["press conference", "hearing", "investigation", "meeting", "interview",
          "briefing", "summit", "announcement", "statement release"]

def generate_improved_headline(topic_idx):
    template = random.choice(templates['headline'])
    topic = topic_names[topic_idx]
    keywords = keywords_by_topic[topic_idx]

    return template.format(
        topic=topic.title(),
        keyword=random.choice(keywords).title(),
        keyword2=random.choice([k for k in keywords if k != keywords[0]]).title(),
        org=random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Organization",
        person=random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "Expert"
    )

def generate_improved_article():
    # Select primary topic for article
    primary_topic = random.randint(0, n_topics-1)

    # Ensure we have enough sentences for this topic
    if len(sentences_by_topic[primary_topic]) < 5:
        primary_topic = max(sentences_by_topic.items(), key=lambda x: len(x[1]))[0]

    # Generate headline
    headline = generate_improved_headline(primary_topic)

    # Start building article
    article = [headline, ""]

    # Introduction
    intro_template = random.choice(templates['intro'])
    article.append(intro_template.format(topic=topic_names[primary_topic]))

    # Add 1-2 specific sentences from the topic (RANDOM SELECTION - SIMPLE VERSION)
    topic_sentences = sentences_by_topic[primary_topic]
    main_sentences = random.sample(topic_sentences, min(2, len(topic_sentences)))
    article.extend(main_sentences)
    article.append("")

    # Add a quote
    if topic_sentences:
        quote_template = random.choice(templates['quote'])
        article.append(quote_template.format(
            sentence=random.choice(topic_sentences),
            person=random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "John Smith",
            org=random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Associated Press",
            verb=random.choice(verbs),
            noun=random.choice(nouns),
            event=random.choice(events)
        ))

    # Add a secondary perspective (from related topic)
    related_topics = list(range(n_topics))
    related_topics.remove(primary_topic)
    if related_topics:
        secondary_topic = random.choice(related_topics)
        if sentences_by_topic[secondary_topic]:
            article.append("")
            article.append(random.choice(templates['transition']))
            secondary_sentences = random.sample(sentences_by_topic[secondary_topic],
                                               min(2, len(sentences_by_topic[secondary_topic])))
            article.extend(secondary_sentences)

    # Conclusion
    if topic_sentences:
        article.append("")
        article.append(random.choice(topic_sentences))

    return "\n".join(article)

print("\n" + "="*80)
print("GENERATING IMPROVED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)")
print("="*80)

for i in range(3):
    print(f"\nARTICLE #{i+1}:")
    print("="*50)
    article = generate_improved_article()
    print(article)
    print("="*50)

print("\nDISCLAIMER: These articles are entirely synthetic and generated for research purposes.")
print("They may contain factual inaccuracies and should not be shared as real news.")

Extracting sentences...
Vectorizing sentences...
Fitting LDA model...
Extracting entities...

GENERATING IMPROVED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)

ARTICLE #1:
JUST IN: Doing Using Scandal Rocks the Armenian Government

A recent development regarding does help little has sparked widespread debate.
The Greeks I mentioned who wouldn't talk to me are educated people.
They can quote you a price and tell you where to send your money.

"I wonder how many American-owned companies employ those in Central Ohio?" said ya, who addressed the statement.

Meanwhile, other sources indicate that
Turks and Kurds demand the right to return to their lands, to determine their own future as a nation in their own homeland.
So It will be nice if Yigal will tell us: 1.

So another strong game from Tommy Soderstrom who hadn't been tested much in his last couple of starts.

ARTICLE #2:
BREAKING: Use Year Let - Day Thing Raises Concerns

New information has emerged about use year let that dem

### Explanation per Function :

#### Extraction to Generate

In [ ]:
# Extract all high-quality sentences
print("Extracting sentences...")
all_sentences = []
for text in cleaned_texts:
    doc = nlp(text)
    for sent in doc.sents:
        sent_text = sent.text.strip()
        # Filter sentences that are complete and of reasonable length
        if len(sent_text.split()) >= 5 and len(sent_text.split()) <= 30:
            if sent_text[-1] in '.!?':
                if not re.search(r'(^\W|^\d+\.|^>|^I think|^I believe|^However|^But|^And|^Or|^So,|^Also,)', sent_text):
                    all_sentences.append(sent_text)

Extracting sentences...


#### Vectorization & Topic Modeling

In [ ]:
print("Vectorizing sentences...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2,
    max_features=1000,
    stop_words='english'
)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

print("Fitting LDA model...")
n_topics = 15
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42,
    batch_size=128,
    n_jobs=-1
)
lda_output = lda.fit_transform(tfidf_matrix)
topic_assignments = lda_output.argmax(axis=1)

Vectorizing sentences...
Fitting LDA model...


#### Grouping Sentences by Topic

In [ ]:
# Group sentences by topic
sentences_by_topic = defaultdict(list)
for i, topic_idx in enumerate(topic_assignments):
    sentences_by_topic[topic_idx].append(all_sentences[i])

#### Extracting the main keywords from each topic

In [ ]:
# Extract keywords for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
keywords_by_topic = {}

for topic_idx, topic in enumerate(lda.components_):
    top_indices = topic.argsort()[:-11:-1]  # Get top 10 keywords
    top_keywords = [feature_names[i] for i in top_indices]
    keywords_by_topic[topic_idx] = top_keywords

# Define topic names based on keywords
topic_names = {}
for topic_idx, keywords in keywords_by_topic.items():
    topic_names[topic_idx] = ' '.join(keywords[:3])

#### Building Entity Database

In [ ]:
# Create entity database for more realistic entity mentions
entity_types = {
    'PERSON': set(),
    'ORG': set(),
    'GPE': set(),  # Countries, cities
    'DATE': set()
}

print("Extracting entities...")
# Process a subset to save time
for text in random.sample(cleaned_texts, min(1000, len(cleaned_texts))):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entity_types and len(ent.text) > 1:
            entity_types[ent.label_].add(ent.text)

Extracting entities...


#### Template and Helpers

In [ ]:
# Template sentences for different parts of an article
templates = {
    'headline': [
        "BREAKING: {topic} - {keyword} {keyword2} Raises Concerns",
        "ALERT: New Developments in {topic} as {org} Announces {keyword}",
        "EXCLUSIVE: {person} Reveals Critical Information About {topic}",
        "{topic} CRISIS: What {org} Isn't Telling You About {keyword}",
        "JUST IN: {keyword} {keyword2} Scandal Rocks {org}"
    ],
    'intro': [
        "A recent development regarding {topic} has sparked widespread debate.",
        "New information has emerged about {topic} that demands attention.",
        "In a stunning revelation, details about {topic} have come to light.",
        "Reports are circulating about unprecedented events related to {topic}."
    ],
    'quote': [
        "\"{sentence}\" said {person}, who {verb} the {noun}.",
        "\"{sentence}\" according to {person} from {org}.",
        "{person} stated, \"{sentence}\" This comes after {org} {verb} the {noun}.",
        "\"{sentence}\" {person} explained during the recent {event}."
    ],
    'transition': [
        "Meanwhile, other sources indicate that",
        "This development comes at a time when",
        "Experts suggest that this could mean",
        "Analysis of the situation reveals",
        "Critics argue that these events demonstrate"
    ]
}

verbs = ["addressed", "questioned", "investigated", "analyzed", "evaluated",
         "criticized", "supported", "endorsed", "reviewed", "examined"]
nouns = ["situation", "statement", "report", "findings", "allegations",
         "evidence", "controversy", "development", "policy", "decision"]
events = ["press conference", "hearing", "investigation", "meeting", "interview",
          "briefing", "summit", "announcement", "statement release"]

# =====================================================================================
# 7. GENERATION FUNCTIONS (SIMPLE VERSION - RANDOM SENTENCES)
# =====================================================================================
def generate_improved_headline(topic_idx):
    template = random.choice(templates['headline'])
    topic = topic_names[topic_idx]
    keywords = keywords_by_topic[topic_idx]

    return template.format(
        topic=topic.title(),
        keyword=random.choice(keywords).title(),
        keyword2=random.choice([k for k in keywords if k != keywords[0]]).title(),
        org=random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Organization",
        person=random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "Expert"
    )

def generate_improved_article():
    # Select primary topic for article
    primary_topic = random.randint(0, n_topics-1)

    # Ensure we have enough sentences for this topic
    if len(sentences_by_topic[primary_topic]) < 5:
        primary_topic = max(sentences_by_topic.items(), key=lambda x: len(x[1]))[0]

    # Generate headline
    headline = generate_improved_headline(primary_topic)

    # Start building article
    article = [headline, ""]

    # Introduction
    intro_template = random.choice(templates['intro'])
    article.append(intro_template.format(topic=topic_names[primary_topic]))

    # Add 1-2 specific sentences from the topic (RANDOM SELECTION - SIMPLE VERSION)
    topic_sentences = sentences_by_topic[primary_topic]
    main_sentences = random.sample(topic_sentences, min(2, len(topic_sentences)))
    article.extend(main_sentences)
    article.append("")

    # Add a quote
    if topic_sentences:
        quote_template = random.choice(templates['quote'])
        article.append(quote_template.format(
            sentence=random.choice(topic_sentences),
            person=random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "John Smith",
            org=random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Associated Press",
            verb=random.choice(verbs),
            noun=random.choice(nouns),
            event=random.choice(events)
        ))

    # Add a secondary perspective (from related topic)
    related_topics = list(range(n_topics))
    related_topics.remove(primary_topic)
    if related_topics:
        secondary_topic = random.choice(related_topics)
        if sentences_by_topic[secondary_topic]:
            article.append("")
            article.append(random.choice(templates['transition']))
            secondary_sentences = random.sample(sentences_by_topic[secondary_topic],
                                               min(2, len(sentences_by_topic[secondary_topic])))
            article.extend(secondary_sentences)

    # Conclusion
    if topic_sentences:
        article.append("")
        article.append(random.choice(topic_sentences))

    return "\n".join(article)

#### Generate Article

In [ ]:
print("\n" + "="*80)
print("GENERATING IMPROVED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)")
print("="*80)

for i in range(3):
    print(f"\nARTICLE #{i+1}:")
    print("="*50)
    article = generate_improved_article()
    print(article)
    print("="*50)

print("\nDISCLAIMER: These articles are entirely synthetic and generated for research purposes.")
print("They may contain factual inaccuracies and should not be shared as real news.")


GENERATING IMPROVED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)

ARTICLE #1:
ALERT: New Developments in Know Said Right as International Technical Allied Signal, Inc. Announces Better

In a stunning revelation, details about know said right have come to light.
Also Bonds is less in need of protection behind him because he is such a good base stealer a walk is a potential double.
Speaking of Roger and ilk, whatever happened to good ol' gln?

"They sure get shrill whenever their belief structure is being shaken. :  :  Kinda reminds you of the BDs, doesn't it? :  :  Jim : Go to hell." said Steve Foster 1-2, who questioned the report.

Meanwhile, other sources indicate that
From memory it has an 80 MB hard drive.
I certainly didn't, but I'd have a hard time arguing against him at this point.

You should have a sketch or detailed drawing of what you plan on doing.

ARTICLE #2:
JUST IN: Space Isn Scandal Rocks Energy

In a stunning revelation, details about did used space have come

## Full Code Generate Hoax (Versi 2 - Advanced)

In [ ]:
print("Extracting sentences by document...")
all_sentences = []
doc_sentence_mapping = []  # Track which sentence belongs to which document

for doc_idx, text in enumerate(cleaned_texts):
    doc = nlp(text)
    doc_sentences = []
    for sent in doc.sents:
        sent_text = sent.text.strip()
        # Filter sentences that are complete and of reasonable length
        if len(sent_text.split()) >= 5 and len(sent_text.split()) <= 30:
            if sent_text[-1] in '.!?':
                if not re.search(r'(^\W|^\d+\.|^>|^I think|^I believe|^However|^But|^And|^Or|^So,|^Also,)', sent_text):
                    all_sentences.append(sent_text)
                    doc_sentences.append(sent_text)
                    doc_sentence_mapping.append(doc_idx)

print("Vectorizing sentences...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2,
    max_features=1000,
    stop_words='english'
)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

print("Fitting LDA model...")
n_topics = 15
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42,
    batch_size=128,
    n_jobs=-1
)
lda_output = lda.fit_transform(tfidf_matrix)
topic_assignments = lda_output.argmax(axis=1)

sentences_by_topic = defaultdict(list)
sentences_by_topic_doc = defaultdict(lambda: defaultdict(list))

for i, topic_idx in enumerate(topic_assignments):
    sentence = all_sentences[i]
    doc_idx = doc_sentence_mapping[i]

    sentences_by_topic[topic_idx].append(sentence)
    sentences_by_topic_doc[topic_idx][doc_idx].append(sentence)

# Extract keywords for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
keywords_by_topic = {}

for topic_idx, topic in enumerate(lda.components_):
    top_indices = topic.argsort()[:-11:-1]  # Get top 10 keywords
    top_keywords = [feature_names[i] for i in top_indices]
    keywords_by_topic[topic_idx] = top_keywords

# Define topic names based on keywords
topic_names = {}
for topic_idx, keywords in keywords_by_topic.items():
    topic_names[topic_idx] = ' '.join(keywords[:3])

# Create entity database for more realistic entity mentions
entity_types = {
    'PERSON': set(),
    'ORG': set(),
    'GPE': set(),  # Countries, cities
    'DATE': set()
}

print("Extracting entities...")
# Process a subset to save time
for text in random.sample(cleaned_texts, min(1000, len(cleaned_texts))):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entity_types and len(ent.text) > 1:
            entity_types[ent.label_].add(ent.text)

# Template sentences for different parts of an article
templates = {
    'headline': [
        "BREAKING: {topic} - {keyword} {keyword2} Raises Concerns",
        "ALERT: New Developments in {topic} as {org} Announces {keyword}",
        "EXCLUSIVE: {person} Reveals Critical Information About {topic}",
        "{topic} CRISIS: What {org} Isn't Telling You About {keyword}",
        "JUST IN: {keyword} {keyword2} Scandal Rocks {org}"
    ],
    'intro': [
        "A recent development regarding {topic} has sparked widespread debate.",
        "New information has emerged about {topic} that demands attention.",
        "In a stunning revelation, details about {topic} have come to light.",
        "Reports are circulating about unprecedented events related to {topic}."
    ],
    'quote': [
        "\"{sentence}\" said {person}, who {verb} the {noun}.",
        "\"{sentence}\" according to {person} from {org}.",
        "{person} stated, \"{sentence}\" This comes after {org} {verb} the {noun}.",
        "\"{sentence}\" {person} explained during the recent {event}."
    ],
    'transition': [
        "Meanwhile, other sources indicate that",
        "This development comes at a time when",
        "Experts suggest that this could mean",
        "Analysis of the situation reveals",
        "Critics argue that these events demonstrate"
    ]
}

verbs = ["addressed", "questioned", "investigated", "analyzed", "evaluated",
         "criticized", "supported", "endorsed", "reviewed", "examined"]
nouns = ["situation", "statement", "report", "findings", "allegations",
         "evidence", "controversy", "development", "policy", "decision"]
events = ["press conference", "hearing", "investigation", "meeting", "interview",
          "briefing", "summit", "announcement", "statement release"]

def generate_improved_headline(topic_idx, person=None, org=None):
    template = random.choice(templates['headline'])
    topic = topic_names[topic_idx]
    keywords = keywords_by_topic[topic_idx]
    keyword_choice = random.sample(keywords, 2) if len(keywords) >= 2 else [keywords[0], keywords[0]]
    org = org or (random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Organization")
    person = person or (random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "Expert")
    return template.format(
        topic=topic.title(),
        keyword=keyword_choice[0].title(),
        keyword2=keyword_choice[1].title(),
        org=org,
        person=person
    )

def choose_consistent_entities():
    person = random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "John Smith"
    org = random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Associated Press"
    gpe = random.choice(list(entity_types['GPE'])) if entity_types['GPE'] else "USA"
    return person, org, gpe

def generate_improved_article(topic_idx=None):
    # Pilih topik utama
    if topic_idx is None:
        topic_idx = random.randint(0, n_topics-1)
    topic_docs = sentences_by_topic_doc[topic_idx]
    # Pilih dokumen dengan kalimat terbanyak
    doc_idx = max(topic_docs, key=lambda k: len(topic_docs[k]))
    doc_sentences = topic_docs[doc_idx]
    # Pilih 4-5 kalimat berurutan agar koheren
    if len(doc_sentences) > 6:
        start = random.randint(0, len(doc_sentences)-5)
        main_sentences = doc_sentences[start:start+5]
    else:
        main_sentences = doc_sentences

    # Pilih entity konsisten
    person, org, gpe = choose_consistent_entities()

    # Headline
    headline = generate_improved_headline(topic_idx, person, org)
    article = [headline, ""]

    # Intro
    intro_template = random.choice(templates['intro'])
    article.append(intro_template.format(topic=topic_names[topic_idx]))

    # Paragraf utama (SEQUENTIAL SENTENCES - ADVANCED VERSION)
    article.extend(main_sentences)
    article.append("")

    # Kutipan dari kalimat yang mengandung kata 'said', 'stated', atau acak dari paragraf
    quote_candidates = [s for s in main_sentences if re.search(r'said|stated|explained|noted', s, re.IGNORECASE)]
    chosen_quote = random.choice(quote_candidates) if quote_candidates else random.choice(main_sentences)
    quote_template = random.choice(templates['quote'])
    article.append(quote_template.format(
        sentence=chosen_quote,
        person=person,
        org=org,
        verb=random.choice(verbs),
        noun=random.choice(nouns),
        event=random.choice(events)
    ))

    # Transisi ke topik sekunder
    related_topics = list(range(n_topics))
    related_topics.remove(topic_idx)
    secondary_topic = random.choice(related_topics)
    secondary_docs = sentences_by_topic_doc[secondary_topic]
    if secondary_docs:
        sec_doc_idx = max(secondary_docs, key=lambda k: len(secondary_docs[k]))
        sec_doc_sentences = secondary_docs[sec_doc_idx]
        if len(sec_doc_sentences) > 3:
            start = random.randint(0, len(sec_doc_sentences)-2)
            secondary_sentences = sec_doc_sentences[start:start+2]
        else:
            secondary_sentences = sec_doc_sentences
        article.append("")
        article.append(random.choice(templates['transition']))
        article.extend(secondary_sentences)

    # Penutup
    article.append("")
    article.append("Further updates will be provided as the situation develops.")

    return "\n".join(article)

print("\n" + "="*80)
print("GENERATING ADVANCED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)")
print("="*80)

for i in range(3):
    print(f"\nARTICLE #{i+1}:")
    print("="*50)
    article = generate_improved_article()
    print(article)
    print("="*50)

print("\nDISCLAIMER: These articles are entirely synthetic and generated for research purposes.")
print("They may contain factual inaccuracies and should not be shared as real news.")

Extracting sentences by document...
Vectorizing sentences...
Fitting LDA model...
Extracting entities...

GENERATING ADVANCED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)

ARTICLE #1:
JUST IN: Didn Said Scandal Rocks Space Technology a.k.a

A recent development regarding know said right has sparked widespread debate.
How do I know who's who and what's what?
Lyuda said that she was Russian, they said, we'll let you go, we aren't touching the Russians, go.
I can't remember my supervisor's telephone number, but something had to be done.
He didn't have any idea what was going on.
He's the head of our administration.

"Lyuda said that she was Russian, they said, we'll let you go, we aren't touching the Russians, go." said Kolly Gyshlag, who addressed the policy.

Critics argue that these events demonstrate
They only managed to run up there and grab something one time.
I tried to call my boss.

Further updates will be provided as the situation develops.

ARTICLE #2:
ALERT: New Develo

### Explanation per Function

#### Extraction to Generate

In [ ]:
print("Extracting sentences by document...")
all_sentences = []
doc_sentence_mapping = []  # Track which sentence belongs to which document

for doc_idx, text in enumerate(cleaned_texts):
    doc = nlp(text)
    doc_sentences = []
    for sent in doc.sents:
        sent_text = sent.text.strip()
        # Filter sentences that are complete and of reasonable length
        if len(sent_text.split()) >= 5 and len(sent_text.split()) <= 30:
            if sent_text[-1] in '.!?':
                if not re.search(r'(^\W|^\d+\.|^>|^I think|^I believe|^However|^But|^And|^Or|^So,|^Also,)', sent_text):
                    all_sentences.append(sent_text)
                    doc_sentences.append(sent_text)
                    doc_sentence_mapping.append(doc_idx)

Extracting sentences by document...


#### Vectorization & Topic Modeling

In [ ]:
print("Extracting sentences by document...")
all_sentences = []
doc_sentence_mapping = []  # Track which sentence belongs to which document

for doc_idx, text in enumerate(cleaned_texts):
    doc = nlp(text)
    doc_sentences = []
    for sent in doc.sents:
        sent_text = sent.text.strip()
        # Filter sentences that are complete and of reasonable length
        if len(sent_text.split()) >= 5 and len(sent_text.split()) <= 30:
            if sent_text[-1] in '.!?':
                if not re.search(r'(^\W|^\d+\.|^>|^I think|^I believe|^However|^But|^And|^Or|^So,|^Also,)', sent_text):
                    all_sentences.append(sent_text)
                    doc_sentences.append(sent_text)
                    doc_sentence_mapping.append(doc_idx)

Extracting sentences by document...


#### Grouping Sentences by Topic

In [ ]:
sentences_by_topic = defaultdict(list)
sentences_by_topic_doc = defaultdict(lambda: defaultdict(list))

for i, topic_idx in enumerate(topic_assignments):
    sentence = all_sentences[i]
    doc_idx = doc_sentence_mapping[i]

    sentences_by_topic[topic_idx].append(sentence)
    sentences_by_topic_doc[topic_idx][doc_idx].append(sentence)

#### Extracting the main keywords from each topic

In [ ]:
# Extract keywords for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
keywords_by_topic = {}

for topic_idx, topic in enumerate(lda.components_):
    top_indices = topic.argsort()[:-11:-1]  # Get top 10 keywords
    top_keywords = [feature_names[i] for i in top_indices]
    keywords_by_topic[topic_idx] = top_keywords

# Define topic names based on keywords
topic_names = {}
for topic_idx, keywords in keywords_by_topic.items():
    topic_names[topic_idx] = ' '.join(keywords[:3])

#### Building Entity Database

In [ ]:
# Create entity database for more realistic entity mentions
entity_types = {
    'PERSON': set(),
    'ORG': set(),
    'GPE': set(),  # Countries, cities
    'DATE': set()
}

print("Extracting entities...")
# Process a subset to save time
for text in random.sample(cleaned_texts, min(1000, len(cleaned_texts))):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entity_types and len(ent.text) > 1:
            entity_types[ent.label_].add(ent.text)

Extracting entities...


#### Template and Helpers

In [ ]:
# Template sentences for different parts of an article
templates = {
    'headline': [
        "BREAKING: {topic} - {keyword} {keyword2} Raises Concerns",
        "ALERT: New Developments in {topic} as {org} Announces {keyword}",
        "EXCLUSIVE: {person} Reveals Critical Information About {topic}",
        "{topic} CRISIS: What {org} Isn't Telling You About {keyword}",
        "JUST IN: {keyword} {keyword2} Scandal Rocks {org}"
    ],
    'intro': [
        "A recent development regarding {topic} has sparked widespread debate.",
        "New information has emerged about {topic} that demands attention.",
        "In a stunning revelation, details about {topic} have come to light.",
        "Reports are circulating about unprecedented events related to {topic}."
    ],
    'quote': [
        "\"{sentence}\" said {person}, who {verb} the {noun}.",
        "\"{sentence}\" according to {person} from {org}.",
        "{person} stated, \"{sentence}\" This comes after {org} {verb} the {noun}.",
        "\"{sentence}\" {person} explained during the recent {event}."
    ],
    'transition': [
        "Meanwhile, other sources indicate that",
        "This development comes at a time when",
        "Experts suggest that this could mean",
        "Analysis of the situation reveals",
        "Critics argue that these events demonstrate"
    ]
}

verbs = ["addressed", "questioned", "investigated", "analyzed", "evaluated",
         "criticized", "supported", "endorsed", "reviewed", "examined"]
nouns = ["situation", "statement", "report", "findings", "allegations",
         "evidence", "controversy", "development", "policy", "decision"]
events = ["press conference", "hearing", "investigation", "meeting", "interview",
          "briefing", "summit", "announcement", "statement release"]

def generate_improved_headline(topic_idx, person=None, org=None):
    template = random.choice(templates['headline'])
    topic = topic_names[topic_idx]
    keywords = keywords_by_topic[topic_idx]
    keyword_choice = random.sample(keywords, 2) if len(keywords) >= 2 else [keywords[0], keywords[0]]
    org = org or (random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Organization")
    person = person or (random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "Expert")
    return template.format(
        topic=topic.title(),
        keyword=keyword_choice[0].title(),
        keyword2=keyword_choice[1].title(),
        org=org,
        person=person
    )

def choose_consistent_entities():
    person = random.choice(list(entity_types['PERSON'])) if entity_types['PERSON'] else "John Smith"
    org = random.choice(list(entity_types['ORG'])) if entity_types['ORG'] else "Associated Press"
    gpe = random.choice(list(entity_types['GPE'])) if entity_types['GPE'] else "USA"
    return person, org, gpe

def generate_improved_article(topic_idx=None):
    # Pilih topik utama
    if topic_idx is None:
        topic_idx = random.randint(0, n_topics-1)
    topic_docs = sentences_by_topic_doc[topic_idx]
    # Pilih dokumen dengan kalimat terbanyak
    doc_idx = max(topic_docs, key=lambda k: len(topic_docs[k]))
    doc_sentences = topic_docs[doc_idx]
    # Pilih 4-5 kalimat berurutan agar koheren
    if len(doc_sentences) > 6:
        start = random.randint(0, len(doc_sentences)-5)
        main_sentences = doc_sentences[start:start+5]
    else:
        main_sentences = doc_sentences

    # Pilih entity konsisten
    person, org, gpe = choose_consistent_entities()

    # Headline
    headline = generate_improved_headline(topic_idx, person, org)
    article = [headline, ""]

    # Intro
    intro_template = random.choice(templates['intro'])
    article.append(intro_template.format(topic=topic_names[topic_idx]))

    # Paragraf utama (SEQUENTIAL SENTENCES - ADVANCED VERSION)
    article.extend(main_sentences)
    article.append("")

    # Kutipan dari kalimat yang mengandung kata 'said', 'stated', atau acak dari paragraf
    quote_candidates = [s for s in main_sentences if re.search(r'said|stated|explained|noted', s, re.IGNORECASE)]
    chosen_quote = random.choice(quote_candidates) if quote_candidates else random.choice(main_sentences)
    quote_template = random.choice(templates['quote'])
    article.append(quote_template.format(
        sentence=chosen_quote,
        person=person,
        org=org,
        verb=random.choice(verbs),
        noun=random.choice(nouns),
        event=random.choice(events)
    ))

    # Transisi ke topik sekunder
    related_topics = list(range(n_topics))
    related_topics.remove(topic_idx)
    secondary_topic = random.choice(related_topics)
    secondary_docs = sentences_by_topic_doc[secondary_topic]
    if secondary_docs:
        sec_doc_idx = max(secondary_docs, key=lambda k: len(secondary_docs[k]))
        sec_doc_sentences = secondary_docs[sec_doc_idx]
        if len(sec_doc_sentences) > 3:
            start = random.randint(0, len(sec_doc_sentences)-2)
            secondary_sentences = sec_doc_sentences[start:start+2]
        else:
            secondary_sentences = sec_doc_sentences
        article.append("")
        article.append(random.choice(templates['transition']))
        article.extend(secondary_sentences)

    # Penutup
    article.append("")
    article.append("Further updates will be provided as the situation develops.")

    return "\n".join(article)

#### Generate Article

In [ ]:
print("\n" + "="*80)
print("GENERATING ADVANCED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)")
print("="*80)

for i in range(3):
    print(f"\nARTICLE #{i+1}:")
    print("="*50)
    article = generate_improved_article()
    print(article)
    print("="*50)

print("\nDISCLAIMER: These articles are entirely synthetic and generated for research purposes.")
print("They may contain factual inaccuracies and should not be shared as real news.")


GENERATING ADVANCED SYNTHETIC NEWS ARTICLES (FOR RESEARCH PURPOSES ONLY)

ARTICLE #1:
Does Help Little CRISIS: What RGB Isn't Telling You About General

Reports are circulating about unprecedented events related to does help little.
Can I get it in my house?
Bear in mind, too, that we say here applies primarily to ordinary single-family residences.
Where can I get a copy?
Where can I get a copy?
The Canadian Standards Association is an organization made up of various government agencies, power utilities, insurance companies, electrical manufacturers and other organizations.

"Can I get it in my house?" Garry 0 explained during the recent briefing.

Critics argue that these events demonstrate
Can I install a replacement fixture?
What is this nonsense about 3HP on 110V 15A circuits?

Further updates will be provided as the situation develops.

ARTICLE #2:
BREAKING: Time Question Believe - Law Believe Raises Concerns

A recent development regarding time question believe has sparked wides